In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

tf.enable_eager_execution()

from preprocessing import inception_preprocessing
from riptide.data import *
import functools

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
preprocess = functools.partial(inception_preprocessing.preprocess_image, height=224, width=224, is_training=False)

In [4]:
test_dataset = tfrecord_dataset("/data3/imagenet/tfrecords/", "validation", workers =8 , preprocess=preprocess, batch_size=128)

In [5]:
from tensorflow.python.keras.applications.mobilenet import MobileNet
with tf.device("/gpu:0"):
    model = MobileNet(weights='imagenet')

In [6]:
# test read speed
import time
start = time.time()
current = time.time()
with tf.device("/gpu:0"):
    for i, (data, label) in enumerate(tfe.Iterator(test_dataset)):
        z = model(data)
        if i %10 ==0:
            get = time.time()
            print("Step: %d, Time: %f" % (i, get - current))
            current = get
    print(current - start)

Step: 0, Time: 2.430743
Step: 10, Time: 10.231152
Step: 20, Time: 6.623213
Step: 30, Time: 6.245371
Step: 40, Time: 5.822717
Step: 50, Time: 6.322187
Step: 60, Time: 6.282770
Step: 70, Time: 6.628311
Step: 80, Time: 6.199964
Step: 90, Time: 6.663820
Step: 100, Time: 6.421217
Step: 110, Time: 5.955291
Step: 120, Time: 6.579030
Step: 130, Time: 7.056465
Step: 140, Time: 6.693200
Step: 150, Time: 6.573330
Step: 160, Time: 6.780476
Step: 170, Time: 6.468194
Step: 180, Time: 6.450308
Step: 190, Time: 6.563369
Step: 200, Time: 5.679453
Step: 210, Time: 6.650946
Step: 220, Time: 7.112564
Step: 230, Time: 5.786616
Step: 240, Time: 6.510375
Step: 250, Time: 5.848001
Step: 260, Time: 6.767656
Step: 270, Time: 6.781676
Step: 280, Time: 6.551093
Step: 290, Time: 7.075602
Step: 300, Time: 6.378075
Step: 310, Time: 6.093412
Step: 320, Time: 6.060715
Step: 330, Time: 6.328262
Step: 340, Time: 7.123041
Step: 350, Time: 6.546756
Step: 360, Time: 6.712474
Step: 370, Time: 6.266687
Step: 380, Time: 6.129

In [8]:
def evaluate(model, iterator, k=1):
    metric = tfe.metrics.Mean()
    for i, (data, label) in enumerate(iterator):
        with tf.device("/gpu:0"):
            z = model(data)
        metric(tf.cast(tf.nn.in_top_k(z.cpu(), tf.squeeze(label.cpu())-1, k=5), tf.float32))
        if i%10 == 0:
            print(i)
    print(metric.value())

In [ ]:
evaluate(model, tfe.Iterator(test_iterator))